In [149]:
import pandas as pd
import csv

In [210]:
tabla_series = pd.read_csv("./components/info_gustos_series.csv", encoding="utf-8")
tabla_movies = pd.read_csv("./components/info_gustos_peliculas.csv", encoding="utf-8")
tabla_series.columns = ["Nombre", "Carrera", "Edad", "Series"]
tabla_movies.columns = ["Nombre", "Carrera", "Edad", "Pelis"]
# print(tabla_series["Series"])
# print(tabla_movies["Pelis"])
peliculas = tabla_movies["Pelis"].drop_duplicates()
series = tabla_series["Series"].drop_duplicates()
print(peliculas.size + series.size)

184


In [151]:
# Displaying final output format
from IPython.display import display, Markdown, Latex
# LangChain Dependencies
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
# from langchain_community.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langgraph.graph import START, END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import json
import os

In [152]:
# Defining LLM
local_llm = 'llama3'
llama3 = OllamaLLM(model=local_llm, temperature=0)
llama3_json = OllamaLLM(model=local_llm, format='json', temperature=0)

In [153]:
# Generation Prompt

media_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all tv shows and movies, you will be given an array of may tv shows, and your job is to return
    a your response with the format JSON: {{"NameOfTheShow": "genre"}}
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {shows} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["shows"],
)

series_chain = media_analizer | llama3_json | JsonOutputParser()


In [154]:
# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    shows = state["shows"]
    # print(shows, movies)

    # Answer Generation
    generation = series_chain.invoke({"shows": shows})

    return {"generation": generation}

In [155]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    shows: str
    generation : str

In [156]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_edge(START, "generate")
workflow.add_node("generate", generate)
workflow.add_edge("generate", END)

# Compile the workflow
local_agent = workflow.compile()


In [ ]:
def run_agent():
    shows = []
    for serie in series:
        output = local_agent.invoke({"shows": serie})
        # print("=======")
        print(output["generation"])
        shows.append(output["generation"])
    
    for movie in peliculas:
        output = local_agent.invoke({"shows": movie})
        # print("=======")
        print(output["generation"])
        shows.append(output["generation"])

    return shows    

In [160]:
# Test it out!
output = [
  {'Gilmore Girls': 'Drama'},
{'Gossip Girl': 'Drama'},
{'The Vampire Diaries': 'Drama'},
{'Riverdale': 'Teen Drama'},
{'Control-Z': 'Thriller'},
{'Inside Job': 'Documentary'},
{'Gossip Girl': 'Teen Drama'},
{'Friends': 'Comedy'},
{'Virgin River': 'Drama'},
{'Casas de papel': 'Thriller'},
{'South Park': 'Comedy'},
{'Breaking Bad': 'Crime'},
{'Supernatural': 'Fantasy'},
{'La Rosa de Guadalupe': 'Soap Opera'},
{'Yellowstone': 'Western'},
{'Club de Cuervos': 'Drama'},
{'Suits': 'Drama'},
{'El Señor de los Cielos': 'Drama'},
{'Arcane': 'Anime'},
{'Brooklyn Nine-Nine': 'Comedy'},
{'The Boys': 'Action'},
{'Invincible': 'Superhero'},
{'Enchanted': 'Musical Comedy'},
{'La Casa De Papel': 'Crime'},
{'Modern Family': 'Comedy'},
{'Ginny & Georgia': 'Teen Drama'},
{'Love Alarm': 'Romance'},
{'Emily Dickinson': 'Biography'},
{'Bridgerton': 'Romance'},
{'The Glory': 'Drama'},
{'Alice in Borderlands': 'Mystery'},
{'Rick and Morty': 'Science Fiction Comedy'},
{'Avatar: The Legend of Aang': 'Fantasy'},
{'Stranger Things': 'Science Fiction'},
{'The Office': 'Comedy'},
{'The Flash': 'Superhero'},
{'How I Met Your Mother': 'Comedy'},
{'Invincible': 'Superhero'},
{'Black Mirror': 'Science Fiction'},
{'3-Body Problem': 'Science Fiction'},
{'La Gloria': 'Drama'},
{'Reign': 'Historical Drama'},
{'Severance': 'Mystery'},
{'Malcolm in the Middle': 'Comedy'},
{'The Boys': 'Action'},
{'The Office': 'Comedy'},
{'Stranger Things': 'Science Fiction'},
{'The Walking Dead': 'Horror'},
{'The Last of Us': 'Drama'},
{'The Big Bang Theory': 'Comedy'},
{'The Nanny': 'Comedy'},
{'Gotham': 'Crime', 'Action': 'Drama'},
{'Mr. Robot': 'Thriller'},
{'My Demon': 'Horror'},
{'Anne with an E': 'Period Drama'},
{'How to Train Your Dragon': 'Adventure'},
{'Your Name': 'Biography'},
{'Goblin': 'Fantasy'},
{'Atlanta': 'Drama'},
{'The Vince Staples Show': 'Comedy'},
{'Hora-de-Aventura': 'Adventure'},
{'Smiling Friends': 'Comedy'},
{'Full Metal Alchemist: Brotherhood': 'Fantasy'},
{'Peaky Blinders': 'Crime Drama'},
{'Cyberpunk: Edgerunners': 'Science Fiction'},
{'Neon Genesis Evangelion': 'Anime', 'Science Fiction': 'Drama'},
{'Family Guy': 'Comedy'},
{'Solo-Leveling': 'Action'},
{'The Mandalorian': 'Science Fiction'},
{'Moon Knight': 'Action'},
{'Tragones-y-Mazmorras': 'Fantasy'},
{'Lupin III': 'Action'},
{'Tiger & Bunny': 'Superhero'},
{'Sex And The City': 'Comedy'},
{'Chilling Adventures of Sabrina': 'Horror'},
{'Derry Girls': 'Comedy'},
{'Dark': 'Mystery'},
{'Twin Peaks': 'Mystery'},
{'Lost': 'Science Fiction'},
{'Vikings': 'Action'},
{'Better Call Saul': 'Crime Drama'},
{'The Flash': 'Superhero'},
{'Club-de-Cuervos': 'Comedy'},
{'Friends': 'Comedy'},
{'Violet Evergarden': 'Drama'},
{'Está-bien-No-estar-bien': 'Comedy'},
{'Breaking Bad': 'Crime'},
{'The Sopranos': 'Drama'},
{'Better Call Saul': 'Crime Drama'},
{'Shadow and Bone': 'Fantasy'},
{'Problema-de-los-tres-cuerpos': 'Mystery'},
{'Pirates of the Caribbean': 'Adventure'},
{'Finding Nemo': 'Animation'},
{'La Ila Siniestra': 'Thriller'},
{'Un-pequeño-Favor': 'Comedy'},
{'Interstellar': 'Science Fiction'},
{'Gone Girl': 'Thriller'},
{'Teléfono-Negro': 'Thriller'},
{'The Submarine': 'Drama'},
{'Clueless': 'Comedy'},
{"Something's Gotta Give": 'Romantic Comedy'},
{'Princess Diaries 2': 'Comedy'},
{'Juego de Gemelas': 'Soap Opera'},
{'Interstellar': 'Science Fiction'},
{'Coraline': 'Fantasy'},
{'Insidious': 'Horror'},
{'Dune': 'Science Fiction'},
{'Nosferatu': 'Horror'},
{'The Neon Demon': 'Horror'},
{'The Hunger Games': 'Drama'},
{'Black Swan': 'Drama'},
{'Fantasia': 'Musical'},
{'Twilight': 'Fantasy'},
{'Yo-Yo-antes-de-Ti': 'Romance'},
{'Interstellar': 'Science Fiction'},
{'Avatar': 'Science Fiction'},
{'Shadowhunters': 'Fantasy'},
{'Inception': 'Action'},
{'La-La-Land': 'Romantic Musical'},
{'Oppenheimer': 'Biography'},
{'El Efecto Mariposa': 'Drama'},
{'La-La Land': 'Musical'},
{'Shrek': 'Comedy'},
{'Juego de gemelas': 'Soap Opera'},
{'Pride and Prejudice': 'Romance'},
{'Mulan': 'Action'},
{'Parasite': 'Black Comedy'},
{'Superbad': 'Comedy'},
{'Jojo Rabbit': 'Comedy'},
{'Back to the Future': 'Science Fiction'},
{'Jurassic Park': 'Science Fiction'},
{'La La Land': 'Romantic Musical'},
{'Saltburn': 'Drama'},
{'The Hunger Games': 'Science Fiction'},
{'El Hobbit: Un viaje inesperado': 'Fantasy'},
{'El Hobbit': 'Fantasy'},
{'El Hobbit: La Batalla de los Cinco Ejércitos': 'Fantasy'},
{'Get Out': 'Thriller'},
{'Whiplash': 'Drama'},
{'Karate Kid': 'Comedy'},
{'Waves': 'Drama'},
{'Django Unchained': 'Western'},
{'Blade Runner': 'Science Fiction'},
{'Interstellar': 'Science Fiction'},
{'Neon Genesis Evangelion': 'Anime', 'Science Fiction': 'Anime'},
{'The Truman Show': 'Satire'},
{'The Lord of the Rings: The Return of the King': 'Fantasy'},
{'Inglourious Basterds': 'War'},
{'Amarte-Duele': 'Romance'},
{'El Padrino': 'Crime'},
{'Scott Pilgrim vs. the World': 'Comedy'},
{'Gladiator': 'Action'},
{'Cars (2006)': 'Animation'},
{'Pretty Woman': 'Romantic Comedy'},
{'The Proposal': 'Romantic Comedy'},
{'Juego de Amor y Fidelidad': 'Romance'},
{"Bram Stoker's Dracula": 'Horror'},
{'Donnie Darko': 'Mystery'},
{'La Chica del Dragón Tatuado': 'Action'},
{'Requiem for a Dream': 'Drama'},
{'Scott Pilgrim vs. the World': 'Comedy'},
{'Ponyo': 'Anime'},
{'Aniquilación': 'Action'},
{'Tron: Legacy': 'Science Fiction'},
{'Son como niños': 'Comedy'},
{'Amores Perros': 'Drama'},
{'Son Como None': 'Comedy'},
{'Sangre-sudor-y-gloria': 'Drama'},
{'Rendirse jamás': 'Drama'},
{'Alerta-en-lo-profundo': 'Documentary'},
{'El-infierno': 'Drama'},
{'Como-Entrenar-a-tu-Dragón': 'Fantasy'},
{'Juego de Honor': 'Action'},
{'Whisper of the Heart': 'Romance'},
{"Howl's Moving Castle": 'Fantasy'},
{'Arrietty': 'Fantasy'},
{'A-Wisker-Away': 'Fantasy'},
{'Gladiator': 'Action'},
{'Click': 'Comedy'},
{'Donde Están Las Rubias': 'Comedy'},
{'La Era de Hielo': 'Comedy'},
{'La ley de Herodes': 'Thriller'},
{'Untouchable': 'Crime'},
{'Rocky & Bullwinkle': 'Comedy'},
]

In [238]:
print(len(output))
for i in range(len(output)):
  if (len(output[i].keys()) != 1):
    print(output[i].keys(), output[i].values())
    # for key, value in output[i].items():
    #   if (key == "Science Fiction"):
    #     print(output[i])
    #     del output[i]
    #     break




181


In [239]:
# df = pd.DataFrame(output)
# df.to_csv("dict.csv", index=False)
i = 0
with open('dict.csv', 'w') as csv_file:  
  writer = csv.writer(csv_file)
  for o in output:
    for key, value in o.items():
      print(key, value)
      if (key != None and value != None):
        writer.writerow([key, value])

Gilmore Girls Drama
Gossip Girl Drama
The Vampire Diaries Drama
Riverdale Teen Drama
Control-Z Thriller
Inside Job Documentary
Gossip Girl Teen Drama
Friends Comedy
Virgin River Drama
Casas de papel Thriller
South Park Comedy
Breaking Bad Crime
Supernatural Fantasy
La Rosa de Guadalupe Soap Opera
Yellowstone Western
Club de Cuervos Drama
Suits Drama
El Señor de los Cielos Drama
Arcane Anime
Brooklyn Nine-Nine Comedy
The Boys Action
Invincible Superhero
Enchanted Musical Comedy
La Casa De Papel Crime
Modern Family Comedy
Ginny & Georgia Teen Drama
Love Alarm Romance
Emily Dickinson Biography
Bridgerton Romance
The Glory Drama
Alice in Borderlands Mystery
Rick and Morty Science Fiction Comedy
Avatar: The Legend of Aang Fantasy
Stranger Things Science Fiction
The Office Comedy
The Flash Superhero
How I Met Your Mother Comedy
Invincible Superhero
Black Mirror Science Fiction
3-Body Problem Science Fiction
La Gloria Drama
Reign Historical Drama
Severance Mystery
Malcolm in the Middle Comedy

In [ ]:
#References:
# Pandas: https://pandas.pydata.org/docs/getting_started/index.html
# Langchain: https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining 